In [1]:
import os
import sys
import pickle
import json
sys.path.append('../')
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

In [2]:
def get_result_replay(path, epoch=10000):
    res = []
    lateral_shift = total_lateral_shift = lateral_shift_openpilot = 0
    with open(path) as f:
        for line in f:
            if ': yaw_diff:' in line:
                lateral_shift = float(line.strip().split(':')[-3].split(',')[0])
                total_lateral_shift = float(line.strip().split(':')[-2].split(',')[0])
                lateral_shift_openpilot = float(line.strip().split(':')[-1].split(',')[0])

                res.append((lateral_shift, total_lateral_shift, lateral_shift_openpilot))
            
    return pd.DataFrame(res, columns=['lateral_shift', 'total_lateral_shift', 'lateral_shift_openpilot'])

In [3]:
##Beign!!!!!!!!!!

frame = 0
N = 20
M = 20
BASE_DIR = '../logs/comma_benign/'
all_res = []

for sim in tqdm([10]):
    for model_type in ['laneatt', 'scnn', 'ultrafast', 'polylanenet']:
        list_res = []
        for i in range(1, 101):

            df_gt = get_result_replay(BASE_DIR + f'replay_nan_more/scb{i}/replay_comma_more_metric.py_f0.log').tail(N).reset_index(drop=True).head(M)
            df_e2e = get_result_replay(BASE_DIR + f'replay_{model_type}_more/scb{i}/replay_comma_more.py_f0.log').tail(N).reset_index(drop=True).head(M)
            if df_e2e.shape[0] != df_gt.shape[0]:
                print(df_e2e.shape[0], df_gt.shape[0])
                
            metric = 0
            for frame in range(M):
                df_metric  = get_result_replay(BASE_DIR + f'replay_{model_type}_more/scb{i}/replay_comma_more_metric.py_f{frame}.log').tail(N).reset_index(drop=True).head(M)
                if df_metric.shape[0] != df_gt.shape[0]:
                    print(BASE_DIR + f'replay_{model_type}_more/scb{i}/replay_comma_more_metric.py_f{frame}.log')
                    raise Exception(f'{model_type} sc{i} frame={frame}')
                m = (df_metric['total_lateral_shift'] - df_gt['total_lateral_shift']).abs().head(sim).max()
                metric += m / sim

            #metric /= M
            e2e = (df_e2e['total_lateral_shift'] - df_gt['total_lateral_shift']).abs().max()
            res = [frame, i, e2e, metric]
            list_res.append(res)

        df = pd.DataFrame(list_res, columns=['frame', '#', f'{model_type}_e2e', f'{model_type}_metric'])
        
        r, p = pearsonr(df[f'{model_type}_e2e'].values, df[f'{model_type}_metric'].values)
        
        ### To obtain Accuracy and F1 score, please run tusimple-benchmark/example/xxx.ipynb, and use the following code
        
        #df_old = pd.read_csv(f'tusimple-benchmark/example/df_acc_{model_type}.csv')
        #df_old['#'] = df_old['path'].map(lambda x: int(x.split('/')[-3][3:])) 
        #df_old = df_old.groupby('#', as_index=False)[['ACC', 'F1']].mean()
        #df_old = pd.merge(df, df_old, on='#', how='inner')
        #r_acc, p_acc = pearsonr(df_old[f'{model_type}_e2e'].values, df_old[f'ACC'].values)
        #r_f1, p_f1 = pearsonr(df_old[f'{model_type}_e2e'].values, df_old[f'F1'].values)
        
        all_res.append({'model': model_type, 
                               'sim': sim, 
                               'r': r, 'p': p,
                               'E2ELD': df[f'{model_type}_e2e'].mean(),
                                'PSLD': df[f'{model_type}_metric'].mean(),
                        
                               'E2ELD_std': df[f'{model_type}_e2e'].std(),
                                'PSLD_std': df[f'{model_type}_metric'].std(),
                        
                                #'ACC': df_old['ACC'].mean(),
                                 #'F1': df_old['F1'].mean(),
                                #'r_acc': r_acc, 'p_acc': p_acc,
                                #'r_f1': r_f1, 'p_f1': p_f1,
                              })
        print(model_type, r, p)
    break

laneatt 0.3807507261244338 9.310217912632136e-05
scnn 0.9263584340243227 2.392602786779628e-43
ultrafast 0.5433382460588443 5.188695694977497e-09
polylanenet 0.48505225348590464 3.147982692560949e-07



In [4]:
df_res = pd.DataFrame(all_res)
df_res['p*'] = df_res['p'] < 0.05
df_res['p**'] = df_res['p'] < 0.1
pd.options.display.max_rows = 999
df_res

,model,sim,r,p,E2ELD,PSLD,E2ELD_std,PSLD_std,p*,p**
0,laneatt,10,0.380751,9.310218e-05,0.179943,0.025508,0.190516,0.023971,True,True
1,scnn,10,0.926358,2.392603e-43,0.198088,0.041884,0.454840,0.075096,True,True
2,ultrafast,10,0.543338,5.188696e-09,0.208244,0.025631,0.172094,0.022312,True,True
3,polylanenet,10,0.485052,3.147983e-07,0.207906,0.029188,0.189912,0.021556,True,True
